In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import DataReader
from pandas import Series, DataFrame

In [2]:
# 引数の取得
#filedate = '20220315'
import sys
filedate = sys.argv[0]

In [3]:
df_stock = pd.read_csv('data/stock_' + filedate + '.csv', index_col=0, header=[0,1], parse_dates=[0])

In [4]:
df_idnr1 = df_stock.tail(4)

In [5]:
# 階層構造を逆にすることができます。今回は列方向で試します。
df_idnr1 = df_idnr1.swaplevel('Attributes','Symbols',axis=1)

In [6]:
df_idnr1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

In [7]:
# マルチインデックスのカラム名を取得する
list = df_idnr1.columns.get_level_values('Symbols')

In [8]:
# High-Lowの計算結果を新たな列として追加
for i in list: 
    df_idnr1[i, 'diff'] = df_idnr1[i, 'High'] - df_idnr1[i, 'Low']

C:\Users\ricew\AppData\Local\Temp/ipykernel_77664/3198842456.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_idnr1[i, 'diff'] = df_idnr1[i, 'High'] - df_idnr1[i, 'Low']


In [9]:
df_idnr1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

In [10]:
df_sel = df_idnr1.loc[:, (slice(None),'diff')].idxmin()

In [11]:
# 高値と安値の幅が直近で最小になっている銘柄を抽出
df_sel2 = df_sel.unstack(level=1)

In [12]:
# 直近がファイル作成日付と一致する行を抽出
df_sel3 = df_sel2['diff'].dt.date == pd.to_datetime(filedate).date()

In [13]:
# indexを連番に振り直して元のインデックスは削除
df_idnr2 = df_idnr1.tail(2).reset_index(drop=True)

In [14]:
df_idnr2 = df_idnr2.T

In [15]:
df_idnr2 = df_idnr2.pivot_table(index=['Symbols'], columns=['Attributes'])

In [16]:
# 日付の比較Datetimeから日付のみを取得して比較
df_idnr3 = df_idnr2[
            (df_idnr2[(0, 'High')] > df_idnr2[(1, 'High')]) &
            (df_idnr2[(0, 'Low')] < df_idnr2[(1, 'Low')])
            ]

In [17]:
#　前日分の情報は削除
df_idnr4 = df_idnr3.drop(0, axis=1)

In [18]:
# 最上層の列を削除
df_idnr4 = df_idnr4.droplevel(0, axis=1)

In [19]:
df_idnr5 = df_idnr4[df_sel3]

C:\Users\ricew\AppData\Local\Temp/ipykernel_77664/1337780265.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_idnr5 = df_idnr4[df_sel3]


In [20]:
df_idnr5.to_csv('data/idnr4_' + filedate + '.csv')